In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import backend as K



In [ ]:
def preprocess_data(data):
  y = np.asarray(data.iloc[:,0]).reshape(data.shape[0], 1)
  x = np.asarray(data.iloc[:,1:]).reshape(-1, 512)
  return x.tolist(),y.tolist()

def data_reshape(dataX, dataY):
  seq_len = 150
  X = []
  Y = []
  for i in range(0, len(dataY), seq_len):
    x_temp = []
    y_temp = []
    for j in range(0, 150):
      x_temp.append(dataX[i+j])
      # y_temp.append(dataY[i+j]) 
    X.append(x_temp)
    Y.append(dataY[i+j])

    


  return X,Y



In [ ]:
df_train = pd.read_csv('/content/gdrive/MyDrive/sign_language/feature_train.csv')
trainX, trainY = preprocess_data(df_train)
del df_train
trainX.insert(0, trainX[0])
trainY.insert(0, trainY[0])



In [ ]:
df_test = pd.read_csv('/content/gdrive/MyDrive/sign_language/feature_test.csv')
testX, testY = preprocess_data(df_test)
del df_test
testX.insert(0, testX[0])
testY.insert(0, testY[0])



In [ ]:
df_data = pd.read_csv('/content/gdrive/MyDrive/sign_language/feature_2hand.csv')
dataX, dataY = preprocess_data(df_data)
del df_data
dataX.insert(0, dataX[0])
dataY.insert(0, dataY[0])



In [ ]:
dataY = np.array(dataY)
dataY += 42

In [ ]:
trainX.extend(testX)
trainX.extend(dataX)
trainY.extend(testY)
trainY.extend(dataY)
del testX, testY, dataX, dataY


In [ ]:
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
trainY = le.fit_transform(trainY)
X, Y = data_reshape(trainX, trainY)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
X = np.array(X)
Y = np.reshape(Y, (3200,))

In [ ]:
del trainX, trainY

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
model = Sequential()

model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape = (150,512)))
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation='swish'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='softmax'))

[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 150, 512) <dtype: 'float32'>
(None, 64) <dtype: 'float32'>
bidirectional (None, 150, 512) float32
bidirectional_1 (None, 150, 128) float32
dropout (None, 128) float32
flatten (None, 128) float32
dense (None, 128) float32
dropout_1 (None, 100) float32
dense_1 (None, 100) float32


[None, None, None, None, None, None, None]

In [ ]:
import tensorflow.keras.models
from tensorflow.keras.applications import EfficientNetB4
from sklearn.model_selection import StratifiedKFold
batch_size = 8
no_classes = 64
no_epochs = 30
num_folds = 8

accuracy_list = []
loss_list = []


kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_idx = 1

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for train_ids, val_ids in kfold.split(X,Y):


 
  print("Bắt đầu train Fold ", fold_idx)


  model.fit(X[train_ids], Y[train_ids],validation_data=(X[val_ids], Y[val_ids]),
              batch_size=batch_size,
              epochs=no_epochs)
 
  # Test và in kết quả
  scores = model.evaluate(X[val_ids], Y[val_ids], verbose=0)
  print("Đã train xong Fold ", fold_idx)
 
  # Thêm thông tin accuracy và loss vào list
  accuracy_list.append(scores[1] * 100)
  loss_list.append(scores[0])
 
  # Sang Fold tiếp theo
  fold_idx = fold_idx + 1



# In kết quả tổng thể
print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
  print(f'> Fold {i+1} - Loss: {loss_list[i]} - Accuracy: {accuracy_list[i]}%')

print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
  print(f'> Fold {i+1} - Loss: {loss_list[i]} - Accuracy: {accuracy_list[i]}%')
 
print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

Bắt đầu train Fold  1
Epoch 1/30
350/350 [==============================] - 43s 29ms/step - loss: 2.8013 - accuracy: 0.4335 - val_loss: 1.2011 - val_accuracy: 0.6750
Epoch 2/30
350/350 [==============================] - 9s 26ms/step - loss: 1.1334 - accuracy: 0.6689 - val_loss: 0.9282 - val_accuracy: 0.7325
Epoch 3/30
350/350 [==============================] - 9s 25ms/step - loss: 0.9216 - accuracy: 0.7246 - val_loss: 0.8093 - val_accuracy: 0.7425
Epoch 4/30
350/350 [==============================] - 9s 25ms/step - loss: 0.7767 - accuracy: 0.7655 - val_loss: 0.7457 - val_accuracy: 0.7600
Epoch 5/30
350/350 [==============================] - 9s 25ms/step - loss: 0.6724 - accuracy: 0.7938 - val_loss: 0.6277 - val_accuracy: 0.7975
Epoch 6/30
350/350 [==============================] - 9s 25ms/step - loss: 0.6580 - accuracy: 0.7834 - val_loss: 0.6710 - val_accuracy: 0.8025
Epoch 7/30
350/350 [==============================] - 9s 25ms/step - loss: 0.6283 - accuracy: 0.7957 - val_loss: 0.5563

In [ ]:
model.save('/content/gdrive/MyDrive/sign_language/sign_9x.model')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/sign_language/sign_9x.model/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/sign_language/sign_9x.model/assets
